# LSST SNR Limit

We plot the modeled magnitude of type Ia supernova as a function of phase and wavelength and compare it against the predicted LSST single visit depth in the WFD fields. See the [LSST science book](https://arxiv.org/pdf/0912.0201.pdf) for source information on depth values.


In [ ]:
import sys
from copy import deepcopy
from pathlib import Path

import numpy as np
import sncosmo
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator

sys.path.insert(0, '../')
from analysis import models

models.register_sources(force=True)
figure_output_dir = Path('./figures/lsst_snr_limit').resolve()
figure_output_dir.mkdir(exist_ok=True, parents=True)
depths = {'LSSTu': 23.9, 'LSSTg': 25.0, 'LSSTr': 24.7, 
          'LSSTi': 24.0, 'LSSTz': 23.3, 'LSSTy': 22.1}


## General Plotting Functions

We will be plotting alot of contours, so we define a few functions to make our lives easier later on.

In [ ]:
def plot_clabel(cs):
    """Plot labels on figure contours
    
    Args:
        cs: Return of pyplot.contour
    """

    class nf(float):
        def __repr__(self):
            s = f'{self:.1f}'
            return f'{self:.0f}' if s[-1] == '0' else s

    cs.levels = [nf(val) for val in cs.levels]
    plt.clabel(cs, cs.levels, inline=True, fmt='%r', fontsize=10)
    
    
def subplot_cotours(x, y, z, axis, vmin, vmax, limit, **kwargs):
    """Imshow values with contours
    
    Args:
        x     (array): An array of x values
        y     (array): An array of y values
        z     (array): A 2d array of z values
        axis   (Axis): matplotlib axis to plot on
        vmin  (float): Lower limit for imshow
        vmax  (float): Upper limit for imshow
        limit (float): A value indicating the LSST 5 sigma limit
        Any other arguments for ``axis.imshow``
        
    Returns:
        A matplotlib figure
    """

    axis.set_xlim(min(x), max(x))
    axis.set_ylim(min(y), max(y))
    extent = [min(x), max(x), min(y), max(y)]
    
    # Color fill
    im = axis.imshow(
        z, 
        origin='lower', 
        interpolation='bilinear',
        extent=extent, 
        aspect='auto',
        vmin=vmin, 
        vmax=vmax,
        **kwargs
    )

    # Magnitude Contours
    levels = [l for l in np.arange(vmin, vmax) if l != limit]
    contours = axis.contour(
        x, y, z, 
        levels=levels, 
        extent=extent,
        colors='black'
    )

    # 5 sigma depth contour
    lsst_limit = axis.contour(
        x, y, z,  
        levels=[limit], 
        extent=extent,
        colors='red',
        linestyles='--'
    )
    
    plot_clabel(contours)
    contours.collections[0].set_label('Magnitude')
    plot_clabel(lsst_limit)
    lsst_limit.collections[0].set_label(r'LSST 5$\sigma$ depth')

    axis.xaxis.set_major_locator(MultipleLocator(10))
    axis.yaxis.set_major_locator(MultipleLocator(.1))
    axis.yaxis.set_minor_locator(MultipleLocator(.05))
    
    return im


## Magnitude Simulation

We start by simulating aparent magnitudes in LSST bands for a range of phases and redshifts.

In [ ]:
def calculate_magnitude(model, redshift, phase, filt):
    """Calculate magnitudes in a given filter
    
    This function is a wrapper for ``model.bandmag`` and
    returns NAN values for phases / redshifts that would
    otherwise raise a ValueError for being out of the model
    range.
    
    Args:
        model    (Model): An sncosmo model
        redshift (array): An array of redshift values
        phase    (array): An array of phase values
        filt       (str): Name of an scnosmo registered filter
        
    Returns:
        A 2d array of magnitudes for each phase / radshift in the given filter
    """
    
    model = deepcopy(model)
    
    mag = []
    for z in redshift:
        model.set(z=z)
        model.set_source_peakabsmag(-19.0, 'bessellb', 'ab')
        
        try:
            mag.append(model.bandmag(filt, 'ab', phase))
            
        except ValueError:
            mag_this = []
            for p in phase:
                try:
                    mag_this.append(model.bandmag(filt, 'ab', p))

                except ValueError:
                    mag_this.append(np.nan)

            mag.append(mag_this)

    return np.ma.array(mag, mask=np.isnan(mag))


In [ ]:
def plot_magnitude(filter_name, redshift_range, phase_range, sources):
    """Color plot of magnitude vs phase and redshift
    
    Args:
        filter_name      (str): Name of the filter to plot
        redshift_range (tuple): Redshift range parameters
        phase_range    (tuple): Phase range parameters
        sources         (list): List of sncosmo sources or source names
        
    Returns:
        A matplotlib figure
        An array of matplotlib axes
    """

    redshift = np.arange(*redshift_range)
    phase = np.arange(*phase_range)
    fig, axes = plt.subplots(
        1, len(sources), figsize=(7.5 * len(sources), 7.5))

    for source, axis in zip(sources, axes.flatten()):
        sn_model = sncosmo.Model(source)
        magnitude = calculate_magnitude(sn_model, redshift, phase, filter_name)
        im = subplot_cotours(
            x=phase, 
            y=redshift, 
            z=magnitude, 
            axis=axis, 
            limit=depths[filter_name], 
            vmin=18, 
            vmax=30, 
            cmap='Blues_r')
        
        axis.set_title(source, fontsize=14)

    axes[0].set_xlabel('Phase', fontsize=14)
    axes[0].set_ylabel('Redshift', fontsize=14)
    axes[0].legend(loc='lower right')
    axes[1].set_xlabel('Phase', fontsize=14)

    cbar_ax = fig.add_axes([.92, 0.15, 0.05, 0.7])
    fig.colorbar(im, cax=cbar_ax)

    return fig, axes
    

In [ ]:
filter_name = 'LSSTr'
redshift_range = (.001, 1.01, .01)
phase_range = (-20, 100.5, .5)
sources = ['Salt2', 'Hsiao']
plot_magnitude(filter_name, redshift_range, phase_range, sources)
plt.savefig(figure_output_dir / 'salt2_hsiao_mag.pdf')
plt.show()
        

## SNR Estimation with Hsiao

We walk through the derivation of a mathematical mapping from observed magnitude to observed signal to noise. The apparent flux at the depth limit of an LSST visit is given by 

$$ \text{depth} = -2.5log(f_d) + zp $$

Since the single visit depth for LSST is given for a signal to noise ratio (SNR) of 5 we have that

$$ 5 = \frac{f_d}{\sigma f_d} = \frac{f_d}{\sigma f_{\text{sky}} + \sigma f_{\text{other}}} \approx \frac{f_d}{\sigma f_{\text{sky}}} $$

Using the above two equations we can express the SNR of an arbitrary observation as

$$ \frac{f}{f_d} \approx \frac{f}{\sigma f_{\text{sky}}} = 5 \frac{f}{f_d} = 5 \times 10^{\left(\frac{m - zp}{-2.5}\right)} 10^{-\left(\frac{\text{depth} - zp}{-2.5}\right)} = 5 \times 10^\left({\frac{m - \text{depth}}{-2.5}}\right)$$



In [ ]:
def snr(mag, filter_name):
    """Return the SNR for apparent magnitude in a given band
    
    Args:
        mag     (ndarray): An array of magnitude values
        filter_name (str): The name of an LSST filter
        
    Returns:
        An array of SNR values
    """

    return 5 * 10 ** ((mag - depths[filter_name]) / -2.5)


In [ ]:
def plot_snr(redshift_range, phase_range, model):
    """Color plot of SNR vs phase and redshift
    
    Args:
        redshift_range (tuple): Redshift range parameters
        phase_range    (tuple): Phase range parameters
        model          (Model): An sncosmo model
        
    Returns:
        A matplotlib figure
        An array of matplotlib axes
    """

    redshift = np.arange(*redshift_range)
    phase = np.arange(*phase_range)

    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    for band, axis in zip(depths.keys(), axes.flatten()):
        magnitude = calculate_magnitude(model, redshift, phase, band)
        snr_arr = snr(magnitude, band)
        im = subplot_cotours(
            x=phase, 
            y=redshift, 
            z=snr_arr, 
            axis=axis, 
            limit=5, 
            vmin=0, 
            vmax=10, 
            cmap='Blues')

        axis.set_title(band, fontsize=14)

    axis.legend()
    for axis in axes[-1, :]:
        axis.set_xlabel('Phase')

    for axis in axes[:, 0]:
        axis.set_ylabel('Redshift')

    cbar_ax = fig.add_axes([.92, 0.15, 0.05, 0.7])
    fig.colorbar(im, cax=cbar_ax)

    return fig, axes


In [ ]:
plot_snr(redshift_range, phase_range, sncosmo.Model('hsiao'))
plt.savefig(figure_output_dir / 'hsiao_snr.pdf')
plt.show()
        

##  SNR Estimation with Custom Models

In [ ]:
sources = [sncosmo.get_source('CMFGEN', version=v) for v in (1.02, 1.04, 1.4, 1.7)]
plot_magnitude('LSSTr', redshift_range, phase_range, sources)
plt.savefig(figure_output_dir / 'custom_model_mag.pdf')
plt.show()
        

In [ ]:
for source in sources:
    fig, axes = plot_snr(redshift_range, phase_range, sncosmo.Model(source))
    fig.suptitle(r'{} $M_{{\odot}}={}$'.format(source.name, source.version), fontsize=16)
    plt.savefig(figure_output_dir / f'{source.name}_{source.version}_snr.pdf')
    plt.show()
